# Image Colorization with OpenVINO running in DeGirum PySDK

This notebook demonstrates how to colorize images using the Colorization model colorization-v2

Given a grayscale image as input, the model generates colorized version of the image as the output.

### Colorization-v2

The colorization-v2 model is one of the colorization group of models designed to perform image colorization.
Model trained on the ImageNet dataset.
Model consumes L-channel of LAB-image as input and produces predict A- and B-channels of LAB-image as output.


## Prerequisites

In [ ]:
# make sure PySDK and degirum-tools is installed
!pip show degirum || pip install -q degirum
!pip show degirum-tools || pip install -q degirum-tools

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

import degirum as dg
import degirum_tools as dgtools

model_name = 'colorization_v2--256x256_float_openvino_cpu_1'
zoo_url = 'https://cs.degirum.com/degirum/openvino_demos'
img_path = 'test_0.jpg'

zoo = dg.connect(dg.CLOUD, zoo_url, dgtools.get_token())
model = zoo.load_model(model_name)
gray_img = cv2.imread(img_path)

## Run inference on an image

In [ ]:
# Preprocess
h_in, w_in, c_in = gray_img.shape
img_rgb = np.array(gray_img).astype(np.float32) / 255
img_l_rs = np.mean(img_rgb, axis=2) * 100  # model expects input pixel lightness value in range 0-100

# Inference
input = cv2.resize(img_l_rs, (256,256))[np.newaxis, np.newaxis, ...]
res = model(input)
res = res.results[0]['data']
update_res = np.squeeze(res)

# Post-process
out = update_res.transpose((1, 2, 0))
out = cv2.resize(out, (w_in, h_in))
img_lab_out = np.concatenate((img_l_rs[:, :, np.newaxis], out), axis=2)
img_bgr_out = np.clip(cv2.cvtColor(img_lab_out, cv2.COLOR_Lab2RGB), 0, 1)
colorized_image = (cv2.resize(img_bgr_out, (w_in, h_in)) * 255).astype(np.uint8)

# Show results
fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(1, 2, 1)
plt.title("Input", fontsize=20)
ax1.axis("off")
ax2 = fig.add_subplot(1, 2, 2)
plt.title("Colorized", fontsize=20)
ax2.axis("off")
ax1.imshow(gray_img)
ax2.imshow(colorized_image)
plt.show()